### Importing Required Modules

In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras as keras
import matplotlib.pyplot as plt

print("imported all modules succesfully 🎉")

imported all modules succesfully 🎉


## Using the MoveNet model 
Used for extracting coordinates body keypoints, relative to the center of the detected body.
[Link](https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/3) for the MoveNet model

In [2]:
interpreter = tf.lite.Interpreter("../models/lite-model_movenet_singlepose_thunder_3.tflite")
interpreter.allocate_tensors()

print("Loaded MoveNet model succesfully 💪")

Loaded MoveNet model succesfully 💪


## Proprocessing Images
The MoveNet singlepose Thunder model requires input image dimesnion of **256x256x3 (height, width, channels)**, so we have to pre-process images and video frames before using them in the model.

In [3]:
def pre_process_input(path):
    input_image = tf.io.read_file(path)
    input_image = tf.compat.v1.image.decode_image(input_image)

    input_image = tf.expand_dims(input_image,axis=0)
    processed_input_image = tf.image.resize_with_pad(input_image,256,256)
    processed_input_image = tf.cast(processed_input_image, dtype=tf.float32)

    path_split = path.split('/')
    img_name = path_split[len(path_split)-1]
    print("Processed",img_name,"✅")
    return processed_input_image

### Working of the pre-processing function:

In [4]:
PATH = "../data/train/tree/girl1_tree087.jpg"

input_image = tf.compat.v1.image.decode_image(tf.io.read_file(PATH))

print("Shape of the input image before processing:")
print(input_image.shape)

processed_input_image = pre_process_input(PATH)

print("Shape of the input image after processing:")
print(processed_input_image.shape)


Shape of the input image before processing:
(300, 300, 3)
Processed girl1_tree087.jpg ✅
Shape of the input image after processing:
(1, 256, 256, 3)


### Getting the body keypoints from MoveNet

In [7]:
def get_keypoints(path):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    img = pre_process_input(path)
    interpreter.set_tensor(0,img)
    interpreter.invoke()
    keypoints = interpreter.get_tensor(output_details[0]['index'])

    # returning keypoints[0][0] to simplify the shape to (17,3)
    return keypoints[0][0]

### Working of the keypoint identifying function

In [10]:
body_keypoints = get_keypoints(path=PATH)

print(body_keypoints)

Processed girl1_tree087.jpg ✅
[[0.0997206  0.47068775 0.76769596]
 [0.08513908 0.49313512 0.8714211 ]
 [0.08453165 0.45753327 0.77890146]
 [0.10777789 0.5186857  0.6903372 ]
 [0.10523403 0.4419055  0.7973572 ]
 [0.2069232  0.5520027  0.92399526]
 [0.21568042 0.4057288  0.8553238 ]
 [0.3101301  0.6196362  0.63666147]
 [0.33885646 0.34500438 0.8025429 ]
 [0.27154016 0.48493916 0.7429699 ]
 [0.2740677  0.43190908 0.70177406]
 [0.45789164 0.51138264 0.8711545 ]
 [0.45013833 0.4181441  0.9334606 ]
 [0.7078261  0.4966217  0.84454787]
 [0.5783176  0.22143488 0.9307568 ]
 [0.9256475  0.4878212  0.79044014]
 [0.5435622  0.44309914 0.77155876]]


### Interpretation of the keypoints

In [17]:
point_names = ["nose", "left eye", "right eye", "left ear", "right ear", "left shoulder", "right shoulder", "left elbow", "right elbow", "left wrist", "right wrist", "left hip", "right hip", "left knee", "right knee", "left ankle", "right ankle"]

print("name\t\tx\t\ty\t\tconfidence")

for i in range(0,17):
    print(point_names[i]+"\t"+str(body_keypoints[i][0])+"\t"+str(body_keypoints[i][1])+"\t"+str(body_keypoints[i][2]))
        

name		x		y		confidence
nose	0.0997206	0.47068775	0.76769596
left eye	0.08513908	0.49313512	0.8714211
right eye	0.08453165	0.45753327	0.77890146
left ear	0.10777789	0.5186857	0.6903372
right ear	0.10523403	0.4419055	0.7973572
left shoulder	0.2069232	0.5520027	0.92399526
right shoulder	0.21568042	0.4057288	0.8553238
left elbow	0.3101301	0.6196362	0.63666147
right elbow	0.33885646	0.34500438	0.8025429
left wrist	0.27154016	0.48493916	0.7429699
right wrist	0.2740677	0.43190908	0.70177406
left hip	0.45789164	0.51138264	0.8711545
right hip	0.45013833	0.4181441	0.9334606
left knee	0.7078261	0.4966217	0.84454787
right knee	0.5783176	0.22143488	0.9307568
left ankle	0.9256475	0.4878212	0.79044014
right ankle	0.5435622	0.44309914	0.77155876


## Collecting all the training data
Click [here](http://download.tensorflow.org/data/pose_classification/yoga_poses.zip) to download the dataset

In [38]:
def get_pose_data(pose_name):
    pose_dir = "../data/train/"+pose_name
    combined_data = list()
    count = 0
    for file_name in os.listdir(pose_dir):
        print(str(count+1),"Processed",file_name,"🟢")
        keypoints = get_keypoints((pose_dir+"/"+file_name))
        combined_data.append(keypoints)
        count+=1
        
    return np.array(combined_data)

In [39]:
tree_data = get_pose_data(pose_name="tree")
dog_data = get_pose_data(pose_name="dog")
warrior_data = get_pose_data(pose_name="warrior")
all_data = tree_data + dog_data + warrior_data

1 Processed girl1_tree086.jpg 🟢
Processed girl1_tree086.jpg ✅
2 Processed girl1_tree087.jpg 🟢
Processed girl1_tree087.jpg ✅
3 Processed girl1_tree088.jpg 🟢
Processed girl1_tree088.jpg ✅
4 Processed girl1_tree089.jpg 🟢
Processed girl1_tree089.jpg ✅
5 Processed girl1_tree090.jpg 🟢
Processed girl1_tree090.jpg ✅
6 Processed girl1_tree094.jpg 🟢
Processed girl1_tree094.jpg ✅
7 Processed girl1_tree096.jpg 🟢
Processed girl1_tree096.jpg ✅
8 Processed girl1_tree099.jpg 🟢
Processed girl1_tree099.jpg ✅
9 Processed girl1_tree100.jpg 🟢
Processed girl1_tree100.jpg ✅
10 Processed girl1_tree101.jpg 🟢
Processed girl1_tree101.jpg ✅
11 Processed girl1_tree102.jpg 🟢
Processed girl1_tree102.jpg ✅
12 Processed girl1_tree106.jpg 🟢
Processed girl1_tree106.jpg ✅
13 Processed girl1_tree108.jpg 🟢
Processed girl1_tree108.jpg ✅
14 Processed girl1_tree114.jpg 🟢
Processed girl1_tree114.jpg ✅
15 Processed girl1_tree115.jpg 🟢
Processed girl1_tree115.jpg ✅
16 Processed girl1_tree118.jpg 🟢
Processed girl1_tree118.jpg ✅
1

In [40]:
all_data.shape

(200, 17, 3)